In [3]:
!pip3 install fastapi uvicorn pdfplumber unstructured sentence-transformers pandas python-multipart


     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     ------------------------------ ------- 786.4/981.5 kB 2.5 MB/s eta 0:00:01
     ---------------------------------------- 981.5/981.5 kB 2.2 MB/s  0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/5.6 MB ? eta -:--:--
   ------- -------------------------------- 1.0/5.6 MB 3.2 MB/s eta 0:00:02
   ----------- ---------------------------- 1.6/5.6 MB 3.0 MB/s eta 0:00:02
   ---------------- ----------------------- 2.4/5.6 MB 3.1 MB/s eta 0:00:02
   ---------------------- ----------------- 3.1/5.6 MB 3.2 MB/s eta 0:00:01
   --------------------------- ------------ 3.9/5.6 MB 3.5 MB/s eta 0:00:01
   ----------------------------------- ---- 5.0/5.6 MB 3.6 MB/s eta 0:00:

  DEPRECATION: Building 'langdetect' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'langdetect'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip3 install pymupdf ollama nltk scikit-learn


   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.7 MB 1.0 MB/s eta 0:00:18
   - -------------------------------------- 0.8/18.7 MB 1.1 MB/s eta 0:00:17
   -- ------------------------------------- 1.3/18.7 MB 1.3 MB/s eta 0:00:14
   --- ------------------------------------ 1.6/18.7 MB 1.4 MB/s eta 0:00:13
   ----- ---------------------------------- 2.4/18.7 MB 1.7 MB/s eta 0:00:10
   ------ --------------------------------- 2.9/18.7 MB 1.9 MB/s eta 0:00:09
   ------ --------------------------------- 3.1/18.7 MB 1.8 MB/s eta 0:00:09
   ------- -------------------------------- 3.4/18.7 MB 1.8 MB/s eta 0:00:09
   ------- -------------------------------- 3.7/18.7 MB 1.7 MB/s eta 0:00:09
   -------- ------------------------------- 3.9/18.7 MB 1.7 MB/s eta 0:00:09
   -------- --------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import fitz
import ollama
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from ipywidgets import FileUpload, Textarea, Button, Output, VBox
from IPython.display import display
import numpy as np

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YT689BU\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [27]:
import streamlit as st
import fitz
import numpy as np
import ollama
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

# Load stopwords
stop_words = set(stopwords.words("english"))

# Extract PDF text
def extract_pdf_text(file):
    doc = fitz.open(stream=file.read(), filetype="pdf")
    pages = [page.get_text() for page in doc]
    return pages

# Extract important keywords
def extract_keywords(question):
    words = question.lower().split()
    keywords = [w for w in words if w not in stop_words and len(w) > 3]
    return keywords

# Retrieve best matching text from PDF using TF-IDF
def retrieve_relevant_text(keywords, pages):
    vectorizer = TfidfVectorizer()
    page_vectors = vectorizer.fit_transform(pages)
    query = " ".join(keywords)
    query_vec = vectorizer.transform([query])
    cosine_sim = (page_vectors * query_vec.T).toarray().flatten()

    top_idx = np.argsort(cosine_sim)[-2:]
    relevant = "\n\n".join([pages[i] for i in top_idx if cosine_sim[i] > 0])
    return relevant if relevant.strip() else "NO_CONTEXT"

# Model call
def generate_response(question, context):
    prompt = f"""
    You are a precise assistant.
    Answer ONLY based on the given PDF context.
    Response must be concise and strictly on topic.
    Do NOT invent anything.
    If the answer isn't found, respond exactly:
    "Not found in document."

    Context:
    {context}

    Question: {question}
    
    Answer:
    """

    result = ollama.chat(
        model="llama3-local",
        messages=[{"role": "user", "content": prompt}],
        options={"num_predict": 120, "temperature": 0.2}
    )
    return result["message"]["content"].strip()


# ----------------------------------------------------------
# ✅ Streamlit UI — Premium Corporate UI
# ----------------------------------------------------------

st.set_page_config(page_title="PDF QA Assistant", page_icon="📄", layout="centered")

st.markdown(
    """
    <style>
        .title-text {
            color: #1F4E79;
            font-size: 32px;
            font-weight: 700;
        }
        .response-box {
            background-color: #F7F9FC;
            padding: 16px;
            border-radius: 10px;
            border-left: 6px solid #1F4E79;
            font-size: 18px;
            color: #000;
        }
        .footer {
            font-size: 13px;
            text-align: center;
            color: gray;
            margin-top: 20px;
        }
    </style>
    """,
    unsafe_allow_html=True,
)

st.markdown("<p class='title-text'>📄 PDF Question Answering (Local AI)</p>", unsafe_allow_html=True)

uploaded_file = st.file_uploader("Upload PDF file", type=["pdf"])

if uploaded_file:
    pages = extract_pdf_text(uploaded_file)
    st.success("✅ PDF processed successfully!")

    question = st.text_input("Ask a question from the document:")

    if st.button("Generate Answer") and question:
        with st.spinner("🔍 Analyzing..."):
            keywords = extract_keywords(question)
            context = retrieve_relevant_text(keywords, pages)

            answer = generate_response(question, context)

        st.markdown(f"<div class='response-box'>{answer}</div>", unsafe_allow_html=True)

st.markdown("<p class='footer'>Powered by llama3-local via Ollama ⚡</p>", unsafe_allow_html=True)


2025-10-27 14:03:13.984 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-27 14:03:13.985 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-27 14:03:13.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-27 14:03:13.987 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-27 14:03:13.987 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-27 14:03:13.988 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-27 14:03:13.989 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-27 14:03:13.989 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()